In [1]:
# %load main.py
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.0.1


In [3]:
# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

Default GPU Device: /gpu:0


In [4]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
   
    tf.saved_model.loader.load(sess,[vgg_tag],vgg_path)
    graph = tf.get_default_graph()
    v_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    v_l_3 = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    v_l_4 = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    v_l_7 = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    
    return v_input, keep, v_l_3, v_l_4, v_l_7
tests.test_load_vgg(load_vgg, tf)

Tests Passed


In [5]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    conv_1x1_7 = tf.layers.conv2d(vgg_layer7_out,num_classes,1,padding='same',
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),
                                  kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
    
    conv_1x1_4 = tf.layers.conv2d(vgg_layer4_out,num_classes,1,padding='same',
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
    
    conv_1x1_3 = tf.layers.conv2d(vgg_layer3_out,num_classes,1,padding='same',
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
    
    
    output = tf.layers.conv2d_transpose(conv_1x1_7,num_classes,4,2,padding='same',
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),
                                       kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
    
    
    output = tf.add(output, conv_1x1_4)
    
    
    output =  tf.layers.conv2d_transpose(output,num_classes,4,2,padding='same',
                                 kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),
                                         kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))

    
    output = tf.add(output,conv_1x1_3)
    
    output = tf.layers.conv2d_transpose(output,num_classes,16,8,padding='same',
                                 kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),
                                        kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
    return output
tests.test_layers(layers)

Tests Passed


In [1]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    
    
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    labels = tf.reshape(correct_label, (-1, num_classes))
    # add  REGULARIZATION_LOSSES
    reg_ws = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    printloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    cross_entropy_loss = printloss + tf.reduce_sum(reg_ws)
    
    train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy_loss)
    

    
    return logits, train_op, cross_entropy_loss,printloss
#tests.test_optimize(optimize)

In [7]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate,printloss):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    import time 
    batch = 0 
    mean_loss = 0
   
    statDf =[]
    for epoch in range(epochs):
        start = time.time()
        for images, labels in get_batches_fn(batch_size):
            _,loss,ploss = sess.run([train_op,cross_entropy_loss,printloss],
                     feed_dict={input_image: images, correct_label: labels, keep_prob: 0.4,learning_rate : 1e-4})
            
            batch+=1
            mean_loss+=ploss
            print("epoch: {}, batch:{}, loss: {:.4f}".format(epoch+1,batch,ploss))
           
        end = time.time()    
        mean_loss = mean_loss/batch
        print("epoch: {},  mean loss: {:.4f}, sec/epoch :{:.2f} ".format(epoch+1, mean_loss,end-start))
        statDf.append([epoch+1,mean_loss])
        batch = 0
        mean_loss = 0
    statDf = np.array(statDf)
    statDf = pd.DataFrame(data=statDf,columns=['epoch','meanloss'])        
           
    return statDf

#tests.test_train_nn(train_nn)

In [ ]:
def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        
        epochs=25
        batch_size=5
        input_image, keep_prob, layer3_out, layer4_out, layer7_out= load_vgg(sess, vgg_path)
        layer_out=layers(layer3_out, layer4_out,layer7_out,num_classes)
        label = tf.placeholder(tf.int32, shape=[None, None, None, num_classes])
        learning_rate = tf.placeholder(tf.float32)
        logits, train_op,loss,ploss=optimize(layer_out,label,learning_rate,num_classes)
        
        
        # TODO: Train NN using the train_nn function
        
        sess.run(tf.global_variables_initializer())
        statDf = train_nn(sess,epochs,batch_size,get_batches_fn,train_op,loss,input_image,label,keep_prob,learning_rate,ploss)
        # TODO: Save inference data using helper.save_inference_samples

        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)
        statDf.plot(x='epoch',y ='meanloss',kind = 'line',grid=True,xticks = statDf.epoch)
        plt.show()
        

        return
    
        #  helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

    
        
        
        


if __name__ == '__main__':
    run()